# **LINMA1702 - Modèles et méthodes d'optimisation**
## *Projet d'optimisation : Première partie*
> *Remarque : Nous avons commencé par l'écriture du rapport PDF en Latex sur base de ce que nous avions codé au préalable. De ce fait, après cela, pour faciliter la rédaction de ce notebook, nous avons utilisé l’outil d’intelligence artificielle ChatGPT afin de retranscrire certains textes issus de notre PDF vers le format Markdown (notamment pour les encadrés, titres et équations). Nous avons bien sûr relu et ajusté ces contenus manuellement après coup pour qu’ils correspondent à ce qu'on avait écrit dans notre rapport PDF.  
> Les codes, modèles et résultats d’optimisation ont quant à eux été entièrement réalisés par nous-mêmes. L’IA n’est intervenue que sur **la retranscription** Markdown, jamais sur les parties techniques.*



---
##  Introduction
Ce projet s’inscrit dans le cadre du cours *LINMA1702 - Modèles et méthodes d’optimisation* et porte sur la planification de la production d’une ligne d’assemblage de vélos électriques. Dans un contexte où les carnets de commande sont connus à l’avance mais sujets à une variabilité importante, l’enjeu est de déterminer une stratégie de production optimale permettant de satisfaire les besoins des clients tout en maîtrisant les coûts de l’entreprise.

La première partie se focalise sur un modèle simplifié où le personnel reste constant tout au long de l’horizon de planification. L’objectif est de modéliser ce problème sous forme d’un programme linéaire continu, en intégrant les différentes possibilités d’ajustement à disposition de l’entreprise : recours aux heures supplémentaires, constitution de stocks, sous-traitance ou encore gestion des retards de livraison.

Ce notebook présente une formulation complète du problème, suivie de son implémentation à l’aide de la bibliothèque Python CVXPY. L’analyse de la solution optimale permet d’évaluer les compromis mis en œuvre par le modèle pour minimiser les coûts globaux. Enfin, une étude de sensibilité est réalisée afin d’observer la robustesse du plan de production face à une perturbation de la demande, ainsi que quelques pistes d’extension du modèle.

L’ensemble du code et des visualisations est intégré dans ce document interactif, facilitant l’interprétation des résultats et la lecture du modèle.


---
### Les imports


In [57]:
import numpy as np
import pandas as pd
import cvxpy as cp
import plotly.graph_objects as go
import time
from statistics import mean

---

##  Question 1 : Formulation, résolution et analyse du modèle et de la solution

Nous modélisons notre problème comme un programme d’optimisation linéaire.  
Ce type de modélisation repose sur la définition :

- d’un vecteur de variables de décision représentant les actions possibles (quantités produites, stockées, sous-traitées, etc.) ;
- d’une fonction objectif qui comptabilise l’ensemble des coûts (production, heures supplémentaires, stockage, sous-traitance, pénalités de retard) ;
- et d’un ensemble de contraintes reflétant les capacités de production, les équilibres entre offre et demande, ainsi que les limitations opérationnelles (stock initial, livraisons autorisées, maximums hebdomadaires, etc.).

On définit donc notre problème d’optimisation comme suit :

$$
\text{minimiser } f(x) \\
\text{sous la contrainte } x \in \Omega
$$

où :

- \( f \) est la fonction objectif ;
- \( x \) est le vecteur des variables de décision ;
- \( $\Omega$ \) est le domaine admissible des variables de décision lié aux contraintes qu’on définit.




---

###  1.1./1.2. Formulation, implémentation et résolution du modèle

Nous modélisons la planification de la production hebdomadaire de vélos sur un horizon de $T = 11$ semaines. L’objectif est de minimiser les coûts globaux tout en répondant à la demande client, en respectant les capacités de production, les possibilités de sous-traitance et les contraintes de stock.



####  Variables de décision

Pour chaque semaine $s = 0, \dots, T-1$ nous introduisons les variables suivantes (dimension temporelle de T):

- $P_s$ : nombre de vélos produits de manière standard à la semaine s 
- $H_s$ : nombre d’heures supplémentaires effectuées à la semaine s
- $ST_s$ : nombre de vélos sous-traités à la semaine s  
- $R1_s$ : nombre de vélos retardés d'1 semaine à la semaine s
- $R2_s$ : nombre de vélos retardés de 2 semaines à la semaine s

Pour pouvoir gérer aisément les stock initiaux et finaux, et l'équilibre entre les différentes semaines, on introduit une autre variable quant à elle définit pour $s = 0, \dots, T$ (dimension temporelle de $T$+1) :
- $S_s$ : stock de vélos à la fin de la semaine s+1 (car $S_0$ fait référence au stock initial imposé)


In [58]:
T=11
P = cp.Variable(T, nonneg=True)
H = cp.Variable(T, nonneg=True)
ST = cp.Variable(T, nonneg=True)
S = cp.Variable(T + 1, nonneg=True)
R1 = cp.Variable(T, nonneg=True)
R2 = cp.Variable(T, nonneg=True)

####  Paramètres

- $T$ : horizon de plannification (nombre de semaines)
- $D_s$ : demande en semaine $s$
- $t_a$ : durée d’assemblage d’un vélo (en minutes)
- $n_o$ : nombre d’ouvriers
- $h_r$ : heures rémunérées par ouvrier et par semaine
- $h_p^{\max}$ : durée maximale dédiée à l'assemblage de vélos par ouvrier par semaine
- $c_o$ : coût horaire par ouvrier
- $c_{HS}$ : coût d'une heure supplémentaire
- $h_s^{\max}$ : nombre maximal d’heures supplémentaires par ouvrier par semaine
- $c_m$ : coût des matériaux par vélo
- $c_{ST}$ : coût de la sous-traitance par vélo
- $n_{st}^{\max}$ : maximum de vélos sous-traitables par semaine
- $c_s$ : coût de stockage par vélo et par semaine  
- $p_1$, $p_2$ : pénalités pour retards de 1 et 2 semaines
- $S_{in}$ : stock initial imposé

In [59]:
demande = np.array([2000, 2000, 2000, 3000, 4000, 2000, 1500, 1500, 2500, 3000, 3000])
duree_assemblage = 15
stock_initial = 250
nb_ouvriers = 16
cout_horaire = 40
nb_heures_remunerees = 38
cout_heure_sup = 55
nb_max_heure_sup = 5
cout_stockage = 5
penalite_1semaine = 10
penalite_2semaines = 15
cout_materiaux = 10
cout_sous_traitant = 25
nb_max_sous_traitant = 500
nb_max_heures_classique = 35

####  Contraintes

**Bilan hebdomadaire** :
A chaque semaine on a un équilibre entre production totale, retards et besoins :

$$
S_{s-1} + P_s + ST_s + \frac{60}{t_a} H_s + R1_s + R2_s = D_s + S_s + R1_{s-1} + R2_{s-2}
$$

(avec \( R1_0 = R2_0 = R2_1 = 0 \) par convention)

**Capacité de production** :
Cette contrainte garantit que le temps alloué à la production ne dépasse pas la capacité horaire totale des ouvriers sur une semaine (hors heures supp.) :

$$
P_s \cdot t_a \leq n_o \cdot 35 \cdot 60
$$

**Heures supplémentaires** :
On impose que les heures supplémentaires par ouvrier ne dépassent pas la limite autorisée :

$$
H_s \leq h_s^{\max} \cdot n_o
$$

**Sous-traitance** :
Le nombre de vélos sous-traités doit respecter un seuil maximal :

$$
ST_s \leq n_{st}^{\max}
$$

**Retards bornés** :
On limite les retards en fonction de la demande des périodes précédentes :
$$
R1_s \leq D_{s-1} \quad \text{(pour } s \geq 1 \text{)}, \quad R2_s \leq D_{s-2} - R1_{s-1} \quad \text{(pour } s \geq 2 \text{)}
$$

**Retards non transférables en fin d'horizon** :
Les retards ne doivent pas être reportés au-delà de l'horizon :
$$
R1_{T-1} = 0, \quad R2_{T-1} = R2_{T-2} = 0
$$

**Stock initial et final** :
Nous imposons que le stock final soit identique au stock initial :

$$
S_0 = S_T = S_{in}
$$

In [60]:
contraintes = [S[0] == stock_initial, S[T] == stock_initial,
                R1[T-1] == 0, R2[T-1] == 0, R2[T-2] == 0]

for s in range(T):
        production_totale = P[s] + ST[s] + (H[s] * 60 / duree_assemblage)
        demande_totale = demande[s] + (R1[s-1] if s >= 1 else 0) + (R2[s-2] if s >= 2 else 0)
        contraintes += [S[s] + production_totale + R1[s] + R2[s] == demande_totale + S[s+1]]
        contraintes += [P[s] * duree_assemblage <= nb_ouvriers * nb_max_heures_classique * 60,
                        H[s] <= nb_max_heure_sup * nb_ouvriers,
                        ST[s] <= nb_max_sous_traitant]

for s in range(1, T):
        contraintes.append(R1[s] <= demande[s-1])
for s in range(2, T):
        contraintes.append(R2[s] <= demande[s-2] - R1[s-1])


####  Fonction objectif

On cherche à minimiser le coût total :

$$
\min \sum_{s=0}^{T-1} \left[
c_m \cdot \left( P_s + \frac{60}{t_a} H_s \right)
+ c_{HS} \cdot H_s
+ c_{ST} \cdot ST_s
+ c_s \cdot S_{s+1}
+ p_1 \cdot R1_s
+ p_2 \cdot R2_s
\right]
+ n_o \cdot h_r \cdot c_o \cdot T
$$

où chaque terme représente un poste de coût :

- $c_m \cdot \left( P_s + \frac{60}{t_a} H_s \right)$ : coût des matériaux pour la production standard et via les heures supplémentaires.
- $c_{HS} \cdot H_s$ : coût des heures supplémentaires effectuées à la semaine $s$.
- $c_{ST} \cdot ST_s$ : coût de sous-traitance pour $ST_s$ vélos.
- $c_s \cdot S_{s+1}$ : coût de stockage des vélos non livrés à la fin de la semaine $s$ (on ne considère pas le stock initial).
- $p_1 \cdot R1_s$ : pénalité pour vélos livrés avec une semaine de retard.
- $p_2 \cdot R2_s$ : pénalité pour vélos livrés avec deux semaines de retard.
- $n_o \cdot h_r \cdot c_o \cdot T$ : salaire fixe total des ouvriers sur les $T$ semaines.








In [61]:
cout_total = cp.sum(
        cout_materiaux * (P + (H * 60 / duree_assemblage)) +
        cout_heure_sup * H +
        cout_sous_traitant * ST +
        cout_stockage * S[1:] +
        penalite_1semaine * R1 +
        penalite_2semaines * R2
    ) + nb_ouvriers * nb_heures_remunerees * cout_horaire * T

---
#### Résolution
Une fois la modélisation effectuée, nous appelons un solveur linéaire qui implémente la méthode du dual, en l’occurrence **GLPK**, pour obtenir la solution optimale. La fonction `vecteurs_1_2()` renvoie les vecteurs demandés dans l'énoncé du projet.

In [62]:
problem = cp.Problem(cp.Minimize(cout_total), contraintes)
problem.solve(solver=cp.GLPK)

plan_velos = P.value
plan_heure_supp = H.value
plan_sous_traitant = ST.value
plan_stockage = S.value[1:]
R1_val = R1.value
R2_val = R2.value

def vecteurs_1_2() :

    return plan_velos, plan_heure_supp, plan_sous_traitant, plan_stockage
print(vecteurs_1_2())

(array([1750., 2240., 2240., 2240., 2240., 2240., 2200., 2240., 2240.,
       2240., 2240.]), array([ 0.,  0.,  0., 70., 80.,  0.,  0.,  0.,  0., 80., 80.]), array([  0.,   0.,   0.,   0., 500.,   0.,   0.,   0.,   0., 150., 500.]), array([  0., 240., 480.,   0.,   0.,   0.,   0., 740., 480., 190., 250.]))


---

###  1.3. Illustration et analyse de notre solution optimale
Le **coût total minimal** obtenu est :

**591 620 €**  
(dont **coût variable** : 324 100 €, **salaire fixe des ouvriers** : 267 520 €)






In [63]:
plan_velos, plan_heure_supp, plan_sous_traitant, plan_stockage = vecteurs_1_2()

weeks = list(range(1, T + 1))
prod_total = plan_velos + plan_sous_traitant + (plan_heure_supp * 60 / duree_assemblage)
satis_heure = demande - R1_val - R2_val

df_all = pd.DataFrame({
    "Semaine": weeks,
    "Demande": demande,
    "Prod_standard": plan_velos,
    "Heures_sup": plan_heure_supp,
    "Sous_traitance": plan_sous_traitant,
    "Stock": plan_stockage,
    "R1": R1_val,
    "R2": R2_val,
    "Prod_totale": prod_total,
    "Livraison à l'heure": satis_heure
})

In [64]:
df_affiche = df_all[[
    "Semaine", "Prod_standard", "Heures_sup", "Sous_traitance",
    "Prod_totale", "Demande", "Livraison à l'heure", "R1", "R2", "Stock"
]]

df_affiche.style.set_properties(**{
    'text-align': 'center'
}).set_table_styles([
    dict(selector='th', props=[('text-align', 'center')])
]).set_caption("Résumé hebdomadaire de notre solution optimale")


,Semaine,Prod_standard,Heures_sup,Sous_traitance,Prod_totale,Demande,Livraison à l'heure,R1,R2,Stock
0,1,1750.000000,0.000000,0.000000,1750.000000,2000,2000.000000,0.000000,0.000000,0.000000
1,2,2240.000000,0.000000,0.000000,2240.000000,2000,2000.000000,0.000000,0.000000,240.000000
2,3,2240.000000,0.000000,0.000000,2240.000000,2000,2000.000000,0.000000,0.000000,480.000000
3,4,2240.000000,70.000000,0.000000,2520.000000,3000,3000.000000,0.000000,0.000000,0.000000
4,5,2240.000000,80.000000,500.000000,3060.000000,4000,3060.000000,240.000000,700.000000,0.000000
5,6,2240.000000,0.000000,0.000000,2240.000000,2000,2000.000000,0.000000,0.000000,0.000000
6,7,2200.000000,0.000000,0.000000,2200.000000,1500,1500.000000,0.000000,0.000000,0.000000
7,8,2240.000000,0.000000,0.000000,2240.000000,1500,1500.000000,0.000000,0.000000,740.000000
8,9,2240.000000,0.000000,0.000000,2240.000000,2500,2500.000000,0.000000,0.000000,480.000000
9,10,2240.000000,80.000000,150.000000,2710.000000,3000,3000.000000,0.000000,0.000000,190.000000


Le tableau ci-dessus présente un résumé semaine par semaine des résultats obtenus.

In [65]:
df_all["Satisfaction"] = df_all["Demande"] - df_all["R1"] - df_all["R2"]
fig = go.Figure()
fig.add_trace(go.Bar(x=df_all["Semaine"], y=df_all["Satisfaction"], name="Livraison à l'heure", marker_color="lightgreen"))
fig.add_trace(go.Bar(x=df_all["Semaine"], y=df_all["R1"], name="Retard 1 semaine", marker_color="gold"))
fig.add_trace(go.Bar(x=df_all["Semaine"], y=df_all["R2"], name="Retard 2 semaines", marker_color="tomato"))
fig.add_trace(go.Scatter(x=df_all["Semaine"], y=df_all["Demande"], name="Demande", mode="lines+markers", line=dict(color="black", dash="dash")))
fig.update_layout(barmode='stack', title="Livraison hebdomadaire avec retards causés", xaxis_title="Semaine [-]", yaxis_title="Nombre de vélos [-]", template="simple_white")
fig.show()

Le graphique ci-dessus vérifie que la demande est systématiquement satisfaite, soit à l’heure (barres vertes), soit avec un léger retard (barres jaunes et rouges), tout en maintenant un équilibre global semaine après semaine.

In [66]:
total_standard = df_all["Prod_standard"].sum()
total_heure_sup = (df_all["Heures_sup"] * 60 / duree_assemblage).sum()
total_sous_traitance = df_all["Sous_traitance"].sum()
labels = ["Production standard", "Heures supplémentaires", "Sous-traitance"]
values = [total_standard, total_heure_sup, total_sous_traitance]
colors = ["royalblue", "orange", "mediumpurple"]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3, marker=dict(colors=colors))])
fig.update_layout(title="Répartition des moyens de production", template="simple_white")
fig.show()

In [67]:
df_all["Cout_mat"] = cout_materiaux * (df_all["Prod_standard"] + (df_all["Heures_sup"] * 60 / duree_assemblage))
df_all["Cout_HS"] = cout_heure_sup * df_all["Heures_sup"]
df_all["Cout_ST"] = cout_sous_traitant * df_all["Sous_traitance"]
df_all["Cout_stock"] = cout_stockage * df_all["Stock"]
df_all["Cout_retards"] = penalite_1semaine * df_all["R1"] + penalite_2semaines * df_all["R2"]
salaire_fixe = nb_ouvriers * nb_heures_remunerees * cout_horaire * T
total_couts = {
    "Matériaux": df_all["Cout_mat"].sum(),
    "Heures supplémentaires": df_all["Cout_HS"].sum(),
    "Sous-traitance": df_all["Cout_ST"].sum(),
    "Stockage": df_all["Cout_stock"].sum(),
    "Retards": df_all["Cout_retards"].sum(),
    "Salaire fixe": salaire_fixe
}
fig = go.Figure(data=[go.Pie(labels=list(total_couts.keys()), values=list(total_couts.values()), hole=0.3)])
fig.update_layout(title="Répartition globale des coûts", template="simple_white")
fig.show()

Ces diagrammes circulaires permettent de relier les choix du modèle aux coûts induits. La production interne est privilégiée, ce qui explique la forte proportion de salaires et de matériaux dans le coût total.

In [68]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_all["Semaine"], y=df_all["Prod_totale"], name="Production totale", mode="lines+markers", line=dict(color="royalblue")))
fig.add_trace(go.Scatter(x=df_all["Semaine"], y=df_all["Demande"], name="Demande", mode="lines+markers", line=dict(color="crimson", dash="dash")))
fig.update_layout(title="Production vs Demande", xaxis_title="Semaine [-]", yaxis_title="Nombre de vélos [-]", template="simple_white")
fig.show()

La production suit une trajectoire relativement régulière malgré la demande qui présente une forte variabilité. Cela est dû en partie à une gestion lissée, visant à éviter les pics de charge coûteux ou irréalistes.


### Observations

**Satisfaction maîtrisée de la demande.**  
Le premier graphe montre que de manière générale, l’entreprise satisfait la demande en livrant dans les temps même si certains retards peuvent apparaître (semaine 5). Cela illustre un équilibre efficace entre production anticipée, utilisation du stock et retards.

**Stabilité de la production**  
La 3ème figure nous montre l’utilité de ce projet : répondre à des demandes variables, en produisant de manière stable et à moindre coût. En effet, même si la demande subit des pics, la production hebdomadaire ne varie que très peu. Une bonne utilisation du stock et des retards permet d’arriver à cette stabilité.

**Répartition des ressources.**  
Sur les diagrammes circulaires, on peut observer que l’entreprise cherche à maximiser l’utilisation des moyens internes. Cette volonté est logique car se rendre trop dépendant des sous-traitants ou des heures supplémentaires des employés fragilise l’entreprise qui est alors plus vulnérable aux variations du marché. En effet, la production standard représente la grande majorité du volume produit, ce qui limite les surcoûts liés aux heures supplémentaires et à la sous-traitance. Cela explique la structure du coût total dû essentiellement aux salaires fixes et les matériaux.

**Retards et stock**  
Le recours au stock et aux retards agit comme sécurité pour absorber les pics de demande. Ces mécanismes sont utilisés avec parcimonie, preuve que le modèle parvient à satisfaire les contraintes tout en limitant les pénalités. Cela est crucial dans un contexte où la demande peut fluctuer fortement.

---
### Performances du solver
Au-delà de la qualité de la solution, il est important d'évaluer la éfficacité de résolution du modèle. Pour cela, nous avons mesuré les temps d'exécution du solveur sur 10 éxecutions consécutives. Les résultats obtenus sont présentés ci-dessous.

In [69]:
def resoudre_modele():
    problem.solve(solver=cp.GLPK)
temps_resolution = []

for _ in range(10):
    start = time.time()
    resoudre_modele()
    end = time.time()
    temps_resolution.append(end - start)


moyenne = mean(temps_resolution)

fig = go.Figure()
fig.add_trace(go.Bar(x=[f"Run {i+1}" for i in range(10)], y=temps_resolution, name="Temps", marker_color="lightblue"))
fig.add_trace(go.Scatter(x=[f"Run {i+1}" for i in range(10)], y=[moyenne]*10, mode="lines", name=f"Moyenne = {moyenne:.4f}s", line=dict(color="red", dash="dash")))
fig.update_layout(title="Temps de résolution du modèle", xaxis_title="Exécution [-]", yaxis_title="Temps [s]", template="simple_white")
fig.show()


Les temps de résolution sont quasiment instantanés. Le solveur GLPK fournit donc une solution de manière rapide et stable, avec une variabilité négligeable entre les exécutions. Cela traduit donc une formulation efficace de notre modèle, il est donc bien conditionné.

---

##  Question 2 : Variation de la demande prévue : Procédure d'évaluation et analyse

Dans cette section, nous cherchons à anticiper l'effet d'une petite variation du vecteur de demande hebdomadaire sur la solution optimale obtenue par notre modèle, sans avoir à relancer la résolution complète du programme. Cela permet de faire une analyse de sensibilité efficace, utile pour évaluer la robustesse du plan de production face à de légers imprévus.

L’idée est d’utiliser les outils d’analyse post-optimale, et en particulier les valeurs duales, pour estimer comment la valeur optimale évolue si la demande change légèrement.


###  2.1. Déscription de la procédure

> ⚠️ **Rappel théorique — Analyse post-optimale sur le second membre**

Considérons un programme linéaire standard :

$$ 
\min \, c^T x \quad \text{tel que} \quad Ax = b, \quad x \geq 0 
$$

Soit $B$ une base optimale du système, alors :

- La solution optimale du primal est $x^* = (x_B^*, x_N^*) = (B^{-1}b, 0)$  
- La solution optimale du dual est $y^* = B^{-T}c_B$  
- La valeur optimale est $z^* = c^T x^* = c_B^T B^{-1}b$  

Si on modifie le second membre selon $b \to b + \varepsilon \cdot \delta b$, alors tant que la base $B$ reste optimale, la nouvelle valeur optimale est donnée par :

$$ 
z^*(\varepsilon) = z^* + \varepsilon \cdot y^{*T} \delta b 
$$

où $y^*$ est la solution du dual. Chaque composante $y_i^*$ représente le coût marginal associé à la contrainte $i$, c’est-à-dire l’augmentation du coût objectif pour une unité de relâchement sur cette contrainte.

---


#### Procédure adaptée à notre modèle de production

Nous souhaitons analyser l’effet d’une variation de la demande :

$$
\text{demande} \quad \longrightarrow \quad \text{demande} + \varepsilon \cdot \text{delta\_demande}
$$

où :

- delta_demande $\in \mathbb{R}^T$ est un vecteur de perturbation (par exemple : hausse à la semaine 5, ou diminution à la semaine 7),
- $\varepsilon \in [0,1]$ est un petit paramètre scalaire.

#### Étapes de la procédure

1. **Résolution initiale du modèle**  
   On résout une première fois le modèle avec la demande originale. On en extrait les valeurs duales $y^*$ associées aux contraintes de bilan hebdomadaire :

   $$
   \text{stock} + \text{production} + \text{retards} = \text{demande} + \text{stock futur}
   $$

   Ces duals $y^*$ représentent la sensibilité locale du coût optimal à la variation de chaque contrainte.

2. **Accès aux duals dans le code**  
   Avec CVXPY, on récupère ces valeurs duales grâce à l’attribut `.dual_value`. On obtient ainsi :

   $$
   y^* = [y_1^*, y_2^*, \dots, y_T^*] \in \mathbb{R}^T
   $$

3. **Perturbation imposée sur la demande**  
   Le vecteur `delta_demande` est fourni par l’énoncé. Il représente une variation ciblée de la demande sur certaines semaines, par exemple une hausse ponctuelle en semaines 4 et 5, et une baisse en semaine 6 :

   ```python
   delta_demande = [0, 0, 0, +100, +200, -100, 0, 0, 0, 0, 0]

4. **Estimation du coût après variation**  
   Grâce à l’analyse post-optimale, le nouveau coût optimal peut être estimé sans nouvelle résolution :

   $$
   z_{\text{estimé}}(\varepsilon) = z_{\text{initial}} + \varepsilon \cdot y^{*T} \cdot \text{delta\_demande}
   $$



---
###  2.2. Test, comparaison et analyse

Après avoir établi une méthode d’approximation du coût total dans la question précédente, nous proposons ici de la tester numériquement à l’aide des données du projet.

L’objectif est de mesurer, pour différentes valeurs du paramètre $ \varepsilon $, l’écart entre :

- le coût exact, obtenu en résolvant complètement le modèle avec une demande perturbée ;
- et le coût estimé, prédit par l’approximation linéaire basée sur les valeurs duales associées aux contraintes de satisfaction de la demande.

1. On résout le modèle une première fois avec la demande de base, et on extrait la valeur optimale $ f(0) $ ainsi que les valeurs duales $ y^* $ associées aux contraintes de livraison.

2. On calcule le coût approximé pour une demande perturbée de la forme  
   $ \texttt{demande} + \varepsilon \cdot \texttt{delta\_demande} $  
   via la formule linéaire :

$$
f(\varepsilon) \approx f(0) + \varepsilon \cdot y^{*T} \cdot \texttt{delta\_demande}
$$

3. Pour différentes valeurs de $ \varepsilon \in [0, 1] $, on compare :

- $ f_{\text{exact}}(\varepsilon) $, obtenu en résolvant à nouveau complètement le programme ;
- $ f_{\text{linéaire}}(\varepsilon) $, l’estimation basée sur les valeurs duales.


In [70]:
def resolution_dual(demande_modif):
    P = cp.Variable(T, nonneg=True)
    H = cp.Variable(T, nonneg=True)
    ST = cp.Variable(T, nonneg=True)
    S = cp.Variable(T + 1, nonneg=True)
    R1 = cp.Variable(T, nonneg=True)
    R2 = cp.Variable(T, nonneg=True)

    contraintes = [S[0] == stock_initial, S[T] == stock_initial,
                   R1[T-1] == 0, R2[T-1] == 0, R2[T-2] == 0]
    contraintes_egalite = []

    for s in range(T):
        prod = P[s] + ST[s] + (H[s] * 60 / duree_assemblage)
        demande_totale = demande_modif[s] + (R1[s-1] if s >= 1 else 0) + (R2[s-2] if s >= 2 else 0)
        eq = S[s] + prod + R1[s] + R2[s] == demande_totale + S[s+1]
        contraintes.append(eq)
        contraintes_egalite.append(eq)
        contraintes += [P[s] * duree_assemblage <= nb_ouvriers * nb_max_heures_classique * 60,
                        H[s] <= nb_max_heure_sup * nb_ouvriers,
                        ST[s] <= nb_max_sous_traitant]
    for s in range(1, T):
        contraintes.append(R1[s] <= demande_modif[s-1])
    for s in range(2, T):
        contraintes.append(R2[s] <= demande_modif[s-2] - R1[s-1])

    cout = cp.sum(
        cout_materiaux * (P + (H * 60 / duree_assemblage)) +
        cout_heure_sup * H +
        cout_sous_traitant * ST +
        cout_stockage * S[1:] +
        penalite_1semaine * R1 +
        penalite_2semaines * R2
    ) + nb_ouvriers * nb_heures_remunerees * cout_horaire * T

    probleme = cp.Problem(cp.Minimize(cout), contraintes)
    probleme.solve(solver=cp.GLPK)
    duals = np.array([c.dual_value for c in contraintes_egalite])
    return probleme.value, duals

delta_demande = np.array([0, 0, 0, 100, 200, -100, 0, 0, 0, 0, 0])
cout_optimal, duals = resolution_dual(demande)
dy = np.dot(delta_demande, duals)

In [71]:
epsilons = np.linspace(0, 1, 11)
couts_exact, couts_estimes = [], []

for eps in epsilons:
    cout_exact, _ = resolution_dual(demande + eps * delta_demande)
    couts_exact.append(cout_exact)
    couts_estimes.append(cout_optimal + eps*dy)

# Graphe
fig = go.Figure()
fig.add_trace(go.Scatter(x=epsilons, y=couts_exact, mode="lines+markers", name="Coût exact"))
fig.add_trace(go.Scatter(x=epsilons, y=couts_estimes, mode="lines+markers", name="Estimation linéaire", line=dict(dash="dash")))
fig.update_layout(
    title="Coût exact vs estimation linéaire",
    xaxis_title="Epsilon [-]",
    yaxis_title="Coût total [-]",
    template="simple_white",
    legend=dict(
        x=0.1,
        y=1,
        xanchor='left',
        yanchor='top'
    )
)
fig.show()

In [72]:
epsilons = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
couts_exacts = [591620, 592795, 593970, 595270, 596570, 597870]
estimations_lineaires = [591620, 590445, 589270, 588095, 586920, 585745]

ecarts_pourcent = [
    100 * abs(c - e) / c
    for c, e in zip(couts_exacts, estimations_lineaires)
]

df_comparaison = pd.DataFrame({
    "epsilon": epsilons,
    "Coût exact (€)": couts_exacts,
    "Estimation linéaire (€)": estimations_lineaires,
    "Écart (%)": [f"{e:.3f} %" for e in ecarts_pourcent]
})

df_comparaison.style.set_properties(**{
    'text-align': 'center'
}).set_table_styles([
    dict(selector='th', props=[('text-align', 'center')])
]).set_caption("Comparaison coût exact vs estimation linéaire")

,epsilon,Coût exact (€),Estimation linéaire (€),Écart (%)
0,0.000000,591620,591620,0.000 %
1,0.200000,592795,590445,0.396 %
2,0.400000,593970,589270,0.791 %
3,0.600000,595270,588095,1.205 %
4,0.800000,596570,586920,1.618 %
5,1.000000,597870,585745,2.028 %


#### Interprétation

Les résultats présentés dans ce graphe illustrent la différence entre le coût exact et son estimation linéaire lorsque la demande varie progressivement. On constate une très légère divergence entre les deux courbes (l’échelle du graphe est trompeuse) : alors que le coût exact augmente régulièrement avec la hausse de la demande, l’estimation linéaire diminue. L’écart relatif, initialement nul, atteint environ 2 % pour $\varepsilon = 1$. Cette approximation linéaire reste globalement très précise, confirmant ainsi son intérêt pratique pour l’analyse locale autour du point optimal initial.

L’autre chose marquante est que l’estimation linéaire possède une pente négative, alors que la solution exacte montre une augmentation du coût à mesure que $\varepsilon$ augmente. Voici notre explication :

— *Les variables duales correspondent à la variation locale du coût lorsqu’on modifie légèrement les contraintes.* Plus précisément, elles donnent le coût marginal associé à chaque contrainte dans sa formulation initiale. Ainsi, si une contrainte d’équilibre de la forme

$$
S_t + P_t + R_t - D_t - S_{t+1} = 0
$$

est écrite avec un signe négatif devant la demande $D_t$, augmenter cette demande revient à *relâcher* la contrainte (car le terme négatif augmente). Ceci explique pourquoi les variables duales correspondantes peuvent devenir négatives : elles traduisent simplement la manière dont la contrainte est formulée dans le modèle initial.

— *L’estimation linéaire correspond à une approximation locale valable uniquement tant que le régime optimal (ensemble des contraintes actives et non actives) reste inchangé.* Dès que nous modifions suffisamment la demande (ici en augmentant progressivement $\varepsilon$), la solution optimale change de structure : certaines contraintes jusque-là inactives peuvent devenir actives (ou inversement). Cela explique pourquoi, à partir d’un certain seuil, l’approximation linéaire devient erronée et diverge du coût exact.

En conclusion, les variables duales donnent une approximation locale qui reste très précise pour des variations modérées autour de la solution optimale initiale, comme le montrent clairement les résultats obtenus (écart max de 2.028 pour cent).


---

##  Question 3 : Variantes du modèle

###  3.1 Variation du coût d’heure supplémentaire

Si le coût des heures supplémentaires dépend du nombre d’heures effectuées, on introduit deux nouvelles variables :

- $h_{s,1}$ : nombre d’heures supplémentaires effectuées en semaine $s$ au taux de 150%.
- $h_{s,2}$ : nombre d’heures supplémentaires effectuées en semaine $s$ au taux de 200%.

Auxquelles on associe les contraintes suivantes :

$$
0 \leq h_{s,1} \leq 2 \cdot \text{nb\_ouvriers}, \quad 0 \leq h_{s,2} \leq 3 \cdot \text{nb\_ouvriers}, \quad \forall s \in \{0, \dots, T-1\}
$$

Le coût total des heures supplémentaires devient :

$$
\sum_{s=1}^{T} \left( 1.5 \cdot \text{cout\_horaire} \cdot h_{s,1} + 2 \cdot \text{cout\_horaire} \cdot h_{s,2} \right)
$$

Ce modèle reste linéaire car :

- Les nouvelles variables $h_{s,1}$ et $h_{s,2}$ sont continues car elles peuvent prendre n’importe quelle valeur réelle positive dans les bornes imposées par les contraintes.
- La fonction objectif est une somme de termes linéaires.
- Les contraintes restent linéaires.

**Tableau — Cas 1 : Coût strictement croissant**

| Heures supp | Coût par heure       | Coût cumulé             |
|-------------|----------------------|--------------------------|
| 1h          | $1.5 \cdot \text{cout\_horaire}$ | $1.5 \cdot \text{cout\_horaire}$  |
| 2h          | $1.5 \cdot \text{cout\_horaire}$ | $3.0 \cdot \text{cout\_horaire}$  |
| 3h          | $2.0 \cdot \text{cout\_horaire}$ | $5.0 \cdot \text{cout\_horaire}$  |
| 4h          | $2.0 \cdot \text{cout\_horaire}$ | $7.0 \cdot \text{cout\_horaire}$  |
| 5h          | $2.0 \cdot \text{cout\_horaire}$ | $9.0 \cdot \text{cout\_horaire}$  |

---

**Tableau — Cas 2 : Coût non-monotone**

| Heures supp | Coût par heure       | Coût cumulé             |
|-------------|----------------------|--------------------------|
| 1h          | $1.5 \cdot \text{cout\_horaire}$ | $1.5 \cdot \text{cout\_horaire}$  |
| 2h          | $1.5 \cdot \text{cout\_horaire}$ | $3.0 \cdot \text{cout\_horaire}$  |
| 3h          | $1.25 \cdot \text{cout\_horaire}$ | $4.25 \cdot \text{cout\_horaire}$ |
| 4h          | $1.25 \cdot \text{cout\_horaire}$ | $5.5 \cdot \text{cout\_horaire}$  |
| 5h          | $1.25 \cdot \text{cout\_horaire}$ | $6.75 \cdot \text{cout\_horaire}$ |

Si le coût des trois dernières heures supplémentaires tombait à 125% au lieu de 200%, la fonction objectif deviendrait non-monotone. On va introduire des variables binaires, pour segmenter le problème en plusieurs régions linéaires. Cependant le passage d’une région à l’autre introduit une discontinuité, rendant le problème globalement non linéaire de manière continue.

---

### Variables binaires et contraintes

- $z_{s,1} \in \{0,1\}$ : active la première tranche d’heures supplémentaires à 150%.
- $z_{s,2} \in \{0,1\}$ : active la seconde tranche à 125%.
- $h_{s,1}, h_{s,2} \geq 0$ : heures supplémentaires dans chaque tranche.

**Contraintes**

1. **Bornes sur les heures :**

$$
0 \leq h_{s,1} \leq 2 \cdot \text{nb\_ouvriers} \cdot z_{s,1}, \quad 0 \leq h_{s,2} \leq 3 \cdot \text{nb\_ouvriers} \cdot z_{s,2}
$$

2. **Activation séquentielle :**

$$
z_{s,2} \leq z_{s,1}
$$

3. **Total des heures :**

$$
h_s = h_{s,1} + h_{s,2}
$$

4. **Coût des heures supplémentaires :**

$$
\text{Coût} = 1.5 \cdot \text{cout\_horaire} \cdot h_{s,1} + 1.25 \cdot \text{cout\_horaire} \cdot h_{s,2}
$$

---

### Nature du modèle

Ce jeu avec les variables binaires $z_{s,1}$ et $z_{s,2}$ peut introduire des discontinuités en fonction de la combinaison de $z_{s,1}$ et $z_{s,2}$, ce qui va rompre la linéarité. On aura affaire à un programme linéaire en nombres mixtes (MILP). Ce type de problème nécessite une méthode résolution plus complexe, ainsi le solveur utilisé dans notre code ne fonctionnera plus. En effet, avec les différentes combinaisons possibles des variables binaires, l’espace de recherche sera considérablement agrandi et il faudra utiliser des méthodes comme l’algorithme branch-and-bound.

---


### 3.2 Coût des matériaux variable et stockage limité

Si l’on suppose que le coût des matériaux évolue de semaine en semaine, ce coût est donc variable.  
On va devoir alors ajouter deux nouvelles variables (une pour la quantité des matériaux et l’autre pour le stockage des matériaux) à notre modèle tout en modifiant certaines contraintes.

#### Les variables

- \( A_s \) : quantité de matériaux achetés au début de la semaine \( s \)
- \( M_s \) : stock de matériaux en fin de semaine \( s \)

> On sous-entend que ces variables sont exprimées en termes de vélos équivalents.

#### Contraintes

- **Évolution du stock de matériaux** :  
  $$
  M_{s+1} = M_s + A_s - \left( P_s + \frac{60}{t_a} H_s + ST_s \right), \quad \forall s \in \{0, \dots, T-1\}
  $$

- **Capacité maximale de stockage** :  
  $$
  M_s \leq \text{stock\_max\_materiaux}, \quad \forall s \in \{0, \dots, T-1\}
  $$

- **Matériaux disponibles ≥ matériaux utilisés** :  
  $$
  P_s + \frac{60}{t_a} H_s + ST_s \leq M_s + A_s, \quad \forall s
  $$

#### Modification de la fonction objectif

Le coût des matériaux devient :
$$
\sum_{s=0}^{T-1} \text{cout\_materiaux}(s) \cdot A_s
$$

Ce terme remplace l’ancien coût constant des matériaux dans la fonction objectif.

### Interprétation

Ces modifications ajoutent une subtilité au problème le rendant plus réaliste.  
L’entreprise doit maintenant tenir compte de la gestion des matériaux et choisir le moment “optimal” pour minimiser le coût total, tout en s’assurant de ne pas manquer de matériaux pour pouvoir satisfaire la demande.




---
## Conclusion

Après avoir modélisé le problème, implémenté un code Python pour trouver une solution optimale, généré des graphiques représentant la situation et répondu aux autres questions, nous concluons en soulignant l'utilité des méthodes d'optimisation linéaire. Bien que ces problèmes ne couvrent pas toutes les situations du monde réel, ils peuvent néanmoins représenter des cas réalistes et complexes, comme c'est le cas dans ce projet.

Les concepts théoriques et l'application de ces méthodes d'optimisation nous permettent désormais de résoudre efficacement ce type de problème. Ce projet met en avant l'aspect pratique de ce cours et nous encourage à trouver des solutions tout en interprétant correctement les éléments clés, ce qui facilite la prise de décision.

Pour la seconde partie du projet, nous devrons ajouter une dimension importante à notre modélisation, à savoir la gestion du personnel, qui introduira de nouvelles contraintes et variables liées à l'affectation des ouvriers et à la gestion des ressources humaines. De plus, cette partie du projet nécessitera la résolution d'une formulation linéaire en nombres entiers (MILP), ce qui permettra d'intégrer des variables binaires et entières dans notre modèle. Cette évolution du modèle se voudra encore plus réaliste.

---

Pour cette première partie, nous avons réparti les questions et sous-questions, puis effectué une relecture collective de l’ensemble du rapport :

- **Question 1** : Elyas Mzoughi & Diego Asscherickx  
- **Question 2** : Elyas Mzoughi  
- **Question 3** : Diego Asscherickx